In [10]:
import numpy as np
import pandas as pd
import re
from currency_converter import CurrencyConverter
from collections import Counter

In [11]:
df = pd.read_csv("../dat/raw_data.csv")

/home/daniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
df.iloc[df.averageRating < 6] = np.nan

In [13]:
short = df[-100:]
zeros = np.zeros(len(short))
short.loc[:,"endYear"] = zeros

/home/daniel/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [14]:
from IPython.display import display, HTML
short
#mask = (currencies == "₹")
#display(HTML(df[mask].to_html()))

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Budget,Gross US & Canada,Opening weekend US & Canada,Gross worldwide,Rating,Critic reviews,User reviews,directors,writers
273443,tt9894470,movie,VFW,VFW,0.0,2019,0.0,92,"Action,Crime,Horror",6.1,7728.0,NaN,NaN,NaN,"$23,101",Not Rated,106.0,123,nm3976308,"nm2244274,nm2100227"
273444,tt9894660,movie,Baby Done,Baby Done,0.0,2020,0.0,91,Comedy,6.0,1157.0,NaN,NaN,NaN,"$565,790",NaN,29.0,19,nm3555793,nm3089836
273445,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273446,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273447,tt9895024,movie,Heiko's World,Heikos Welt,0.0,2021,0.0,118,"Comedy,Drama,Sport",8.6,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nm8135721,nm8135721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273538,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273539,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273540,tt9916362,movie,Coven,Akelarre,0.0,2020,0.0,92,"Drama,History",6.4,4271.0,"€2,500,000 (estimated)",NaN,NaN,"$317,907",TV-MA,28.0,36,nm1893148,"nm1893148,nm3471432"
273541,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
budget_raw = df["Budget"].to_numpy()
currencies = np.zeros(len(budget_raw), dtype='U10')
value = np.zeros(len(budget_raw), dtype=int)
year = df["startYear"].to_numpy()

for i in range(len(budget_raw)):
    string = budget_raw[i]
    
    if not pd.isna(string):

        value[i] = int("".join(re.findall(r'[\d]+', string)))
        currencies[i] = re.findall(r'[^{\d,\xa0}]+', string)[0]

In [16]:
# USD
df_usd = pd.read_csv("../dat/inflation_data/usd.csv")
np_amount = df_usd["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "$":
        target_year = int(year[i]) if int(year[i]) >= 1910 else 1910
        inflation_factor = current_amount/df_usd[df_usd.year == target_year].amount
        currencies[i] = "usd"
        value[i] = value[i] * inflation_factor
del df_usd

In [17]:
# EUR

c = CurrencyConverter()

df_eur = pd.read_csv("../dat/inflation_data/eur.csv")
np_amount = df_eur["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "€":
        target_year = int(year[i]) if int(year[i]) >= 1996 else 1996
        inflation_factor = current_amount/df_eur[df_eur.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'EUR', 'USD')
        currencies[i] = "usd"
        
        
del df_eur

In [18]:
print(Counter(list(currencies)))

Counter({'': 240840, 'usd': 24023, '₹': 1922, '£': 1508, 'CA$': 1209, 'A$': 523, 'R$': 318, 'SEK': 193, 'NOK': 181, 'DEM': 137, 'RUR': 131, 'CHF': 130, 'FRF': 124, 'DKK': 121, 'ARS': 109, 'HUF': 96, 'FIM': 93, '¥': 92, 'MX$': 92, 'MYR': 90, 'CN¥': 87, 'TRL': 87, 'NZ$': 82, '₩': 73, 'IDR': 67, 'ZAR': 67, 'PLN': 62, 'IRR': 59, '₱': 57, 'EGP': 57, 'NLG': 56, 'BDT': 56, 'ITL': 42, 'CZK': 41, 'ESP': 40, 'SGD': 40, 'ROL': 38, 'PKR': 38, 'UAH': 37, 'NPR': 36, 'THB': 36, 'HK$': 35, 'ISK': 31, 'DOP': 31, '₪': 26, 'LKR': 25, 'NGN': 22, 'PTE': 21, 'HRK': 18, 'BGL': 15, 'NT$': 15, 'RON': 14, 'COP': 14, 'VEB': 11, 'GEL': 10, 'MVR': 9, 'CLP': 9, 'LTL': 8, 'LVL': 8, 'AZM': 8, 'BEF': 7, '₫': 7, 'MNT': 7, 'EEK': 5, 'AED': 5, 'MAD': 5, 'TND': 5, 'AMD': 5, 'SIT': 4, 'GRD': 4, 'YUM': 4, 'MMK': 4, 'TTD': 4, 'IEP': 3, 'GTQ': 3, 'JOD': 3, 'BAM': 3, 'MKD': 3, 'ATS': 2, 'SKK': 2, 'CUP': 2, 'ALL': 2, 'LBP': 2, 'UGX': 2, 'BHD': 2, 'PYG': 2, 'KGS': 2, 'GHC': 2, 'JMD': 1, 'BND': 1, 'OMR': 1, 'BYR': 1, 'IQD': 1, 'N

In [19]:
# INR

c = CurrencyConverter()

df_inr = pd.read_csv("../dat/inflation_data/inr.csv")
np_amount = df_inr["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "₹":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_inr[df_inr.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'INR', 'USD')
        currencies[i] = "usd"
        
        
del df_inr

In [20]:
print(Counter(list(currencies)))

Counter({'': 240840, 'usd': 25945, '£': 1508, 'CA$': 1209, 'A$': 523, 'R$': 318, 'SEK': 193, 'NOK': 181, 'DEM': 137, 'RUR': 131, 'CHF': 130, 'FRF': 124, 'DKK': 121, 'ARS': 109, 'HUF': 96, 'FIM': 93, '¥': 92, 'MX$': 92, 'MYR': 90, 'CN¥': 87, 'TRL': 87, 'NZ$': 82, '₩': 73, 'IDR': 67, 'ZAR': 67, 'PLN': 62, 'IRR': 59, '₱': 57, 'EGP': 57, 'NLG': 56, 'BDT': 56, 'ITL': 42, 'CZK': 41, 'ESP': 40, 'SGD': 40, 'ROL': 38, 'PKR': 38, 'UAH': 37, 'NPR': 36, 'THB': 36, 'HK$': 35, 'ISK': 31, 'DOP': 31, '₪': 26, 'LKR': 25, 'NGN': 22, 'PTE': 21, 'HRK': 18, 'BGL': 15, 'NT$': 15, 'RON': 14, 'COP': 14, 'VEB': 11, 'GEL': 10, 'MVR': 9, 'CLP': 9, 'LTL': 8, 'LVL': 8, 'AZM': 8, 'BEF': 7, '₫': 7, 'MNT': 7, 'EEK': 5, 'AED': 5, 'MAD': 5, 'TND': 5, 'AMD': 5, 'SIT': 4, 'GRD': 4, 'YUM': 4, 'MMK': 4, 'TTD': 4, 'IEP': 3, 'GTQ': 3, 'JOD': 3, 'BAM': 3, 'MKD': 3, 'ATS': 2, 'SKK': 2, 'CUP': 2, 'ALL': 2, 'LBP': 2, 'UGX': 2, 'BHD': 2, 'PYG': 2, 'KGS': 2, 'GHC': 2, 'JMD': 1, 'BND': 1, 'OMR': 1, 'BYR': 1, 'IQD': 1, 'NAD': 1, 'CD

In [21]:
# gbp

c = CurrencyConverter()

df_gbp = pd.read_csv("../dat/inflation_data/gbp.csv")
np_amount = df_gbp["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "£":
        target_year = int(year[i]) if int(year[i]) >= 1910 else 1910
        inflation_factor = current_amount/df_gbp[df_gbp.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'GBP', 'USD')
        currencies[i] = "usd"

        
del df_gbp

In [22]:
print(Counter(list(currencies)))

Counter({'': 240840, 'usd': 27453, 'CA$': 1209, 'A$': 523, 'R$': 318, 'SEK': 193, 'NOK': 181, 'DEM': 137, 'RUR': 131, 'CHF': 130, 'FRF': 124, 'DKK': 121, 'ARS': 109, 'HUF': 96, 'FIM': 93, '¥': 92, 'MX$': 92, 'MYR': 90, 'CN¥': 87, 'TRL': 87, 'NZ$': 82, '₩': 73, 'IDR': 67, 'ZAR': 67, 'PLN': 62, 'IRR': 59, '₱': 57, 'EGP': 57, 'NLG': 56, 'BDT': 56, 'ITL': 42, 'CZK': 41, 'ESP': 40, 'SGD': 40, 'ROL': 38, 'PKR': 38, 'UAH': 37, 'NPR': 36, 'THB': 36, 'HK$': 35, 'ISK': 31, 'DOP': 31, '₪': 26, 'LKR': 25, 'NGN': 22, 'PTE': 21, 'HRK': 18, 'BGL': 15, 'NT$': 15, 'RON': 14, 'COP': 14, 'VEB': 11, 'GEL': 10, 'MVR': 9, 'CLP': 9, 'LTL': 8, 'LVL': 8, 'AZM': 8, 'BEF': 7, '₫': 7, 'MNT': 7, 'EEK': 5, 'AED': 5, 'MAD': 5, 'TND': 5, 'AMD': 5, 'SIT': 4, 'GRD': 4, 'YUM': 4, 'MMK': 4, 'TTD': 4, 'IEP': 3, 'GTQ': 3, 'JOD': 3, 'BAM': 3, 'MKD': 3, 'ATS': 2, 'SKK': 2, 'CUP': 2, 'ALL': 2, 'LBP': 2, 'UGX': 2, 'BHD': 2, 'PYG': 2, 'KGS': 2, 'GHC': 2, 'JMD': 1, 'BND': 1, 'OMR': 1, 'BYR': 1, 'IQD': 1, 'NAD': 1, 'CDF': 1, 'PAB

In [23]:
# cad

c = CurrencyConverter()

df_cad = pd.read_csv("../dat/inflation_data/cad.csv")
np_amount = df_cad["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "CA$":
        target_year = int(year[i]) if int(year[i]) >= 1915 else 1915
        inflation_factor = current_amount/df_cad[df_cad.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'CAD', 'USD')
        currencies[i] = "usd"

        
del df_cad

In [24]:
print(Counter(list(currencies)))

Counter({'': 240840, 'usd': 28662, 'A$': 523, 'R$': 318, 'SEK': 193, 'NOK': 181, 'DEM': 137, 'RUR': 131, 'CHF': 130, 'FRF': 124, 'DKK': 121, 'ARS': 109, 'HUF': 96, 'FIM': 93, '¥': 92, 'MX$': 92, 'MYR': 90, 'CN¥': 87, 'TRL': 87, 'NZ$': 82, '₩': 73, 'IDR': 67, 'ZAR': 67, 'PLN': 62, 'IRR': 59, '₱': 57, 'EGP': 57, 'NLG': 56, 'BDT': 56, 'ITL': 42, 'CZK': 41, 'ESP': 40, 'SGD': 40, 'ROL': 38, 'PKR': 38, 'UAH': 37, 'NPR': 36, 'THB': 36, 'HK$': 35, 'ISK': 31, 'DOP': 31, '₪': 26, 'LKR': 25, 'NGN': 22, 'PTE': 21, 'HRK': 18, 'BGL': 15, 'NT$': 15, 'RON': 14, 'COP': 14, 'VEB': 11, 'GEL': 10, 'MVR': 9, 'CLP': 9, 'LTL': 8, 'LVL': 8, 'AZM': 8, 'BEF': 7, '₫': 7, 'MNT': 7, 'EEK': 5, 'AED': 5, 'MAD': 5, 'TND': 5, 'AMD': 5, 'SIT': 4, 'GRD': 4, 'YUM': 4, 'MMK': 4, 'TTD': 4, 'IEP': 3, 'GTQ': 3, 'JOD': 3, 'BAM': 3, 'MKD': 3, 'ATS': 2, 'SKK': 2, 'CUP': 2, 'ALL': 2, 'LBP': 2, 'UGX': 2, 'BHD': 2, 'PYG': 2, 'KGS': 2, 'GHC': 2, 'JMD': 1, 'BND': 1, 'OMR': 1, 'BYR': 1, 'IQD': 1, 'NAD': 1, 'CDF': 1, 'PAB': 1, 'KWD': 

In [25]:
# aud

c = CurrencyConverter()

df_aud = pd.read_csv("../dat/inflation_data/aud.csv")
np_amount = df_aud["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "A$":
        target_year = int(year[i]) if int(year[i]) >= 1915 else 1915
        inflation_factor = current_amount/df_aud[df_aud.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'AUD', 'USD')
        currencies[i] = "usd"

        
del df_aud

TypeError: cannot convert the series to <class 'float'>

In [ ]:
print(Counter(list(currencies)))

In [ ]:
# sek

c = CurrencyConverter()

df_sek = pd.read_csv("../dat/inflation_data/sek.csv")
np_amount = df_sek["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "SEK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_sek[df_sek.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'SEK', 'USD')
        currencies[i] = "usd"

        
del df_sek

In [ ]:
print(Counter(list(currencies)))

In [ ]:
# nok

c = CurrencyConverter()

df_nok = pd.read_csv("../dat/inflation_data/nok.csv")
np_amount = df_nok["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "NOK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_nok[df_nok.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'NOK', 'USD')
        currencies[i] = "usd"

        
del df_nok

In [ ]:
print(Counter(list(currencies)))

In [ ]:
# zar

c = CurrencyConverter()

df_zar = pd.read_csv("../dat/inflation_data/zar.csv")
np_amount = df_zar["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "R$":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_zar[df_zar.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'ZAR', 'USD')
        currencies[i] = "usd"

        
del df_zar
print(Counter(list(currencies)))

In [ ]:
# dkk

c = CurrencyConverter()

df_dkk = pd.read_csv("../dat/inflation_data/dkk.csv")
np_amount = df_dkk["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "DKK":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_dkk[df_dkk.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'DKK', 'USD')
        currencies[i] = "usd"

        
del df_dkk
print(Counter(list(currencies)))

In [ ]:
# rub

c = CurrencyConverter()

df_rub = pd.read_csv("../dat/inflation_data/rub.csv")
np_amount = df_rub["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "RUR":
        target_year = int(year[i]) if int(year[i]) >= 1993 else 1993
        inflation_factor = current_amount/df_rub[df_rub.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'RUB', 'USD')
        currencies[i] = "usd"

        
del df_rub
print(Counter(list(currencies)))

In [ ]:
# cny

c = CurrencyConverter()

df_cny = pd.read_csv("../dat/inflation_data/cny.csv")
np_amount = df_cny["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "CN¥":
        target_year = int(year[i]) if int(year[i]) >= 1987 else 1987
        inflation_factor = current_amount/df_cny[df_cny.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'CNY', 'USD')
        currencies[i] = "usd"

        
del df_cny
print(Counter(list(currencies)))

In [ ]:
# try

c = CurrencyConverter()

df_try = pd.read_csv("../dat/inflation_data/try.csv")
np_amount = df_try["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "TRL":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_try[df_try.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'TRY', 'USD')
        currencies[i] = "usd"

        
del df_try
print(Counter(list(currencies)))

In [ ]:
# huf

c = CurrencyConverter()

df_huf = pd.read_csv("../dat/inflation_data/huf.csv")
np_amount = df_huf["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "HUF":
        target_year = int(year[i]) if int(year[i]) >= 1973 else 1973
        inflation_factor = current_amount/df_huf[df_huf.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'HUF', 'USD')
        currencies[i] = "usd"

        
del df_huf
print(Counter(list(currencies)))

In [ ]:
# pln

c = CurrencyConverter()

df_pln = pd.read_csv("../dat/inflation_data/jpy.csv")
np_amount = df_pln["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "PLN":
        target_year = int(year[i]) if int(year[i]) >= 1971 else 1971
        inflation_factor = current_amount/df_pln[df_pln.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'PLN', 'USD')
        currencies[i] = "usd"

        
del df_pln
print(Counter(list(currencies)))

In [ ]:
# jpy

c = CurrencyConverter()

df_jpy = pd.read_csv("../dat/inflation_data/jpy.csv")
np_amount = df_jpy["amount"].to_numpy()
current_amount = np_amount[-1]

for i in range(len(budget_raw)):
    if currencies[i] == "¥":
        target_year = int(year[i]) if int(year[i]) >= 1960 else 1960
        inflation_factor = current_amount/df_jpy[df_jpy.year == target_year].amount
        value[i] = c.convert(value[i] * inflation_factor, 'JPY', 'USD')
        currencies[i] = "usd"

        
del df_jpy
print(Counter(list(currencies)))

In [ ]:
pd.read_csv("../dat/inflation_data/jpy.csv")